In [1]:
!pip install torch

     |████████████████████████████████| 676.9MB 110kB/s  eta 0:00:01     |█████                           | 103.5MB 66.7MB/s eta 0:00:09


In [2]:
# import stuff
%load_ext autoreload
%autoreload 2
%matplotlib inline

from random import randint

import numpy as np
import torch

## Load model

In [2]:
# Load model
model_version = 1
MODEL_PATH = "infersent%s.pkl" % model_version
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

SyntaxError: invalid syntax (<ipython-input-2-89d749ac3a6d>, line 2)

In [7]:
# Keep it on CPU or put it on GPU
use_cuda = True
model = model.cuda() if use_cuda else model

In [8]:
# If infersent1 -> use GloVe embeddings. If infersent2 -> use InferSent embeddings.
W2V_PATH = 'glove.840B.300d.txt'
model.set_w2v_path(W2V_PATH)

In [10]:
import pandas as pd
import ast
df = pd.read_csv('tokenized_tweets_new.csv')

In [11]:
df['text_full'] = df["filtered_text"].apply(lambda x: " ".join(ast.literal_eval(x)))

In [12]:
df["times"] = pd.to_datetime((df['time']))
df["year"] = df["times"].apply(lambda x: x.year)
df = df[df["year"]>2017].copy()
df.drop(["Unnamed: 0", "times"], axis=1, inplace=True)

In [13]:
df

,Unnamed: 0.1,id,favorite_count,retweet_count,text,time,user,party,filtered_w_stop_text,filtered_text,text_full,year
0,0,1115311440989184000,12.0,2.0,"Get well soon, Assemblyman. @SeanMRyan149 http...",2019-04-08 17:52:23,NYGovCuomo,1,"['get', 'well', 'soon', 'assemblyman']","['get', 'well', 'soon', 'assemblyman']",get well soon assemblyman,2019
1,1,1115298202092298241,133.0,39.0,This report is deeply alarming and goes agains...,2019-04-08 16:59:47,NYGovCuomo,1,"['this', 'report', 'is', 'deeply', 'alarming',...","['report', 'deeply', 'alarming', 'goes', 'ever...",report deeply alarming goes everything stand a...,2019
2,2,1114890416913092608,96.0,29.0,One of the big wins in the budget was the pass...,2019-04-07 13:59:23,NYGovCuomo,1,"['one', 'of', 'the', 'big', 'wins', 'in', 'the...","['one', 'big', 'wins', 'budget', 'passage', 's...",one big wins budget passage senator jose peral...,2019
3,3,1114667816941162497,169.0,75.0,We have to act quickly to fight climate change...,2019-04-06 23:14:51,NYGovCuomo,1,"['we', 'have', 'to', 'act', 'quickly', 'to', '...","['act', 'quickly', 'fight', 'climate', 'change...",act quickly fight climate change part new york...,2019
4,4,1114546662112923648,129.0,40.0,New York just make some important changes to o...,2019-04-06 15:13:26,NYGovCuomo,1,"['new', 'york', 'just', 'make', 'some', 'impor...","['new', 'york', 'make', 'important', 'changes'...",new york make important changes discovery laws...,2019
5,5,1114223726080536582,125.0,41.0,While Washington threatens the rights of New A...,2019-04-05 17:50:12,NYGovCuomo,1,"['while', 'washington', 'threatens', 'the', 'r...","['washington', 'threatens', 'rights', 'new', '...",washington threatens rights new americans new ...,2019
6,6,1114166661278576640,0.0,10.0,RT @I_LOVE_NY: Craving an adventure? Vote in t...,2019-04-05 14:03:26,NYGovCuomo,1,"['love', 'ny', 'craving', 'an', 'adventure', '...","['love', 'ny', 'craving', 'adventure', 'vote',...",love ny craving adventure vote final round ilo...,2019
7,7,1113975587989798912,136.0,30.0,While the federal administration tries to buil...,2019-04-05 01:24:11,NYGovCuomo,1,"['while', 'the', 'federal', 'administration', ...","['federal', 'administration', 'tries', 'build'...",federal administration tries build wall roll b...,2019
8,8,1113902272801538048,132.0,31.0,"In case you missed it, New York just codified ...",2019-04-04 20:32:51,NYGovCuomo,1,"['in', 'case', 'you', 'missed', 'it', 'new', '...","['case', 'missed', 'new', 'york', 'codified', ...",case missed new york codified key provisions a...,2019
9,9,1113863389397377024,79.0,15.0,A tragedy was averted thanks to the quick acti...,2019-04-04 17:58:21,NYGovCuomo,1,"['a', 'tragedy', 'was', 'averted', 'thanks', '...","['tragedy', 'averted', 'thanks', 'quick', 'act...",tragedy averted thanks quick actions bystander...,2019


In [14]:
!pip install nltk

     |████████████████████████████████| 3.1MB 45.5MB/s eta 0:00:01
  Stored in directory: /home/ubuntu/.cache/pip/wheels/97/8a/10/d646015f33c525688e91986c4544c68019b19a473cb33d3b55
Successfully built nltk


In [15]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
# Load embeddings of K most frequent words
#model.build_vocab_k_words(K=100000)
#once we have the sentences ready
model.build_vocab(df['text_full'])

Found 63389(/106951) words with w2v vectors
Vocab size : 63389


## Encode sentences

In [15]:
# gpu mode : >> 1000 sentences/s
# cpu mode : ~100 sentences/s

In [17]:
embeddings = model.encode(df['text_full'], verbose=True)
print('nb sentences encoded : {0}'.format(len(embeddings)))

Nb words kept : 8632626/8887943 (97.1%)
Speed : 955.8 sentences/s (gpu mode, bsize=64)
nb sentences encoded : 517664


In [17]:
# embeddings = model.encode(sentences, bsize=128, tokenize=False, verbose=True)
# print('nb sentences encoded : {0}'.format(len(embeddings)))

In [19]:
import mxnet as mx

In [20]:
ctx = mx.gpu()

In [21]:
dataset = mx.gluon.data.dataset.ArrayDataset(embeddings, df['party'])

In [22]:
df['party'][df['party'] == -1] = 0

/home/ubuntu/miniconda3/envs/gluon/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
!pip install d2l

In [24]:
import sys
sys.path.insert(0, '..')

import d2l
from mxnet import gluon, init
from mxnet.gluon import loss as gloss, nn

In [25]:
!pip install sklearn

     |████████████████████████████████| 5.4MB 46.1MB/s eta 0:00:01
  Stored in directory: /home/ubuntu/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


In [26]:
from sklearn.model_selection import train_test_split

In [27]:
X_train, X_test, y_train, y_test = train_test_split(embeddings, df['party'], test_size=0.2, random_state=42)

In [28]:
from mxnet import gluon, init, nd, autograd
import time

In [33]:
def train(train_iter, test_iter, net, loss, trainer, ctx, num_epochs, lr_period):
    """Train and evaluate a model."""
    print('training on', ctx)
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, m, start = 0.0, 0.0, 0, 0, time.time()
        if epoch > 0 and epoch % lr_period == 0:
             trainer.set_learning_rate(trainer.learning_rate * .5)
        for i, batch in enumerate(train_iter):
            Xs, ys, batch_size = _get_batch(batch, ctx)
            ls = []
            with autograd.record():
                y_hats = [net(X) for X in Xs]
                ls = [loss(y_hat, y) for y_hat, y in zip(y_hats, ys)]
            for l in ls:
                l.backward()
            trainer.step(batch_size)
            train_l_sum += sum([l.sum().asscalar() for l in ls])
            n += sum([l.size for l in ls])
            train_acc_sum += sum([(y_hat.argmax(axis=1) == y).sum().asscalar()
                                  for y_hat, y in zip(y_hats, ys)])
            m += sum([y.size for y in ys])
        test_acc = evaluate_accuracy(test_iter, net, ctx)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / m, test_acc,
                 time.time() - start))


In [34]:
def _get_batch(batch, ctx):
    """Return features and labels on ctx."""
    features, labels = batch
    if labels.dtype != features.dtype:
        labels = labels.astype(features.dtype)
    return (gutils.split_and_load(features, ctx),
            gutils.split_and_load(labels, ctx), features.shape[0])

def evaluate_accuracy(data_iter, net, ctx=[mx.cpu()]):
    """Evaluate accuracy of a model on the given data set."""
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    acc_sum, n = nd.array([0]), 0
    for batch in data_iter:
        features, labels, _ = _get_batch(batch, ctx)
        for X, y in zip(features, labels):
            y = y.astype('float32')
            acc_sum += (net(X).argmax(axis=1) == y).sum().copyto(mx.cpu())
            n += y.size
        acc_sum.wait_to_read()
    return acc_sum.asscalar() / n

In [35]:
net = nn.HybridSequential()
net.add(nn.Dense(2048, activation='relu'))
net.add(gluon.nn.Dropout(0.25))
net.add(nn.Dense(512, activation='relu'))
net.add(nn.Dense(2))
#     net.initialize(init.Normal(sigma=0.01))
net.initialize(init.Xavier(), ctx=ctx)
net.hybridize()

# net.initialize(init.Normal(sigma=0.01))

batch_size = 128
train_set = gdata.ArrayDataset(X_train, nd.array(y_train))
test_set = gdata.ArrayDataset(X_test, nd.array(y_test))
train_iter = gdata.DataLoader(train_set, batch_size, shuffle=True)
test_iter = gdata.DataLoader(test_set, batch_size)

loss = gloss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})
num_epochs = 100
# d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None,
#               None, trainer)

In [36]:
train(train_iter, test_iter, net, gloss.SoftmaxCrossEntropyLoss(), trainer, ctx, num_epochs, 10)

training on gpu(0)
epoch 1, loss 0.5910, train acc 0.678, test acc 0.690, time 21.6 sec
epoch 2, loss 0.5418, train acc 0.716, test acc 0.712, time 21.5 sec
epoch 3, loss 0.5159, train acc 0.735, test acc 0.739, time 21.5 sec
epoch 4, loss 0.4980, train acc 0.747, test acc 0.675, time 21.5 sec
epoch 5, loss 0.4831, train acc 0.756, test acc 0.716, time 21.5 sec
epoch 6, loss 0.4707, train acc 0.765, test acc 0.765, time 21.5 sec
epoch 7, loss 0.4586, train acc 0.773, test acc 0.753, time 21.5 sec
epoch 8, loss 0.4481, train acc 0.780, test acc 0.772, time 21.6 sec
epoch 9, loss 0.4369, train acc 0.786, test acc 0.775, time 21.8 sec
epoch 10, loss 0.4265, train acc 0.792, test acc 0.773, time 21.7 sec
epoch 11, loss 0.4138, train acc 0.799, test acc 0.745, time 21.6 sec
epoch 12, loss 0.4041, train acc 0.806, test acc 0.772, time 21.6 sec
epoch 13, loss 0.3947, train acc 0.811, test acc 0.740, time 21.5 sec
epoch 14, loss 0.3859, train acc 0.816, test acc 0.782, time 21.5 sec
epoch 15, 

In [37]:
net = nn.HybridSequential()
net.add(nn.Dense(1024, activation='relu'))
net.add(gluon.nn.Dropout(0.25))
net.add(nn.Dense(512, activation='relu'))
net.add(nn.Dense(2))
#     net.initialize(init.Normal(sigma=0.01))
net.initialize(init.Xavier(), ctx=ctx)
net.hybridize()

# net.initialize(init.Normal(sigma=0.01))

batch_size = 128
train_set = gdata.ArrayDataset(X_train, nd.array(y_train))
test_set = gdata.ArrayDataset(X_test, nd.array(y_test))
train_iter = gdata.DataLoader(train_set, batch_size, shuffle=True)
test_iter = gdata.DataLoader(test_set, batch_size)

loss = gloss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})
num_epochs = 100
# d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None,
#               None, trainer)

In [38]:
train(train_iter, test_iter, net, gloss.SoftmaxCrossEntropyLoss(), trainer, ctx, num_epochs, 10)

training on gpu(0)
epoch 1, loss 0.5914, train acc 0.677, test acc 0.704, time 15.8 sec
epoch 2, loss 0.5435, train acc 0.716, test acc 0.725, time 15.7 sec
epoch 3, loss 0.5185, train acc 0.733, test acc 0.738, time 15.8 sec
epoch 4, loss 0.5005, train acc 0.745, test acc 0.731, time 15.8 sec
epoch 5, loss 0.4864, train acc 0.754, test acc 0.757, time 15.8 sec
epoch 6, loss 0.4743, train acc 0.762, test acc 0.688, time 15.6 sec
epoch 7, loss 0.4630, train acc 0.770, test acc 0.769, time 15.6 sec
epoch 8, loss 0.4530, train acc 0.776, test acc 0.757, time 15.6 sec
epoch 9, loss 0.4433, train acc 0.783, test acc 0.764, time 15.7 sec
epoch 10, loss 0.4340, train acc 0.788, test acc 0.757, time 15.9 sec
epoch 11, loss 0.4223, train acc 0.795, test acc 0.751, time 15.7 sec
epoch 12, loss 0.4135, train acc 0.800, test acc 0.780, time 15.7 sec
epoch 13, loss 0.4058, train acc 0.805, test acc 0.776, time 15.7 sec
epoch 14, loss 0.3971, train acc 0.810, test acc 0.779, time 15.7 sec
epoch 15, 

In [39]:
net = nn.HybridSequential()
net.add(nn.Dense(512, activation='relu'))
net.add(gluon.nn.Dropout(0.25))
net.add(nn.Dense(2))
#     net.initialize(init.Normal(sigma=0.01))
net.initialize(init.Xavier(), ctx=ctx)
net.hybridize()

# net.initialize(init.Normal(sigma=0.01))

batch_size = 128
train_set = gdata.ArrayDataset(X_train, nd.array(y_train))
test_set = gdata.ArrayDataset(X_test, nd.array(y_test))
train_iter = gdata.DataLoader(train_set, batch_size, shuffle=True)
test_iter = gdata.DataLoader(test_set, batch_size)

loss = gloss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})
num_epochs = 100
# d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None,
#               None, trainer)

In [40]:
train(train_iter, test_iter, net, gloss.SoftmaxCrossEntropyLoss(), trainer, ctx, num_epochs, 10)

training on gpu(0)
epoch 1, loss 0.5919, train acc 0.679, test acc 0.682, time 14.6 sec
epoch 2, loss 0.5478, train acc 0.713, test acc 0.730, time 14.5 sec
epoch 3, loss 0.5252, train acc 0.730, test acc 0.737, time 14.9 sec
epoch 4, loss 0.5101, train acc 0.740, test acc 0.745, time 15.3 sec
epoch 5, loss 0.4983, train acc 0.749, test acc 0.683, time 15.3 sec
epoch 6, loss 0.4875, train acc 0.756, test acc 0.756, time 15.2 sec
epoch 7, loss 0.4779, train acc 0.762, test acc 0.751, time 15.3 sec
epoch 8, loss 0.4702, train acc 0.767, test acc 0.741, time 15.3 sec
epoch 9, loss 0.4630, train acc 0.772, test acc 0.741, time 15.3 sec
epoch 10, loss 0.4548, train acc 0.777, test acc 0.743, time 15.3 sec
epoch 11, loss 0.4458, train acc 0.784, test acc 0.750, time 15.3 sec
epoch 12, loss 0.4392, train acc 0.787, test acc 0.761, time 15.3 sec
epoch 13, loss 0.4326, train acc 0.792, test acc 0.741, time 15.3 sec
epoch 14, loss 0.4264, train acc 0.796, test acc 0.768, time 15.3 sec
epoch 15, 

## Redefining Train, by adjusting lr decay

In [41]:
def train(train_iter, test_iter, net, loss, trainer, ctx, num_epochs, lr_period):
    """Train and evaluate a model."""
    print('training on', ctx)
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, m, start = 0.0, 0.0, 0, 0, time.time()
        if epoch > 0 and epoch % lr_period == 0:
             trainer.set_learning_rate(trainer.learning_rate * .5)
        for i, batch in enumerate(train_iter):
            Xs, ys, batch_size = _get_batch(batch, ctx)
            ls = []
            with autograd.record():
                y_hats = [net(X) for X in Xs]
                ls = [loss(y_hat, y) for y_hat, y in zip(y_hats, ys)]
            for l in ls:
                l.backward()
            trainer.step(batch_size)
            train_l_sum += sum([l.sum().asscalar() for l in ls])
            n += sum([l.size for l in ls])
            train_acc_sum += sum([(y_hat.argmax(axis=1) == y).sum().asscalar()
                                  for y_hat, y in zip(y_hats, ys)])
            m += sum([y.size for y in ys])
        test_acc = evaluate_accuracy(test_iter, net, ctx)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / m, test_acc,
                 time.time() - start))


In [44]:
net = nn.HybridSequential()
net.add(nn.Dense(512, activation='relu'))
net.add(gluon.nn.Dropout(0.25))
net.add(nn.Dense(2))
#     net.initialize(init.Normal(sigma=0.01))
net.initialize(init.Xavier(), ctx=ctx)
net.hybridize()

# net.initialize(init.Normal(sigma=0.01))

batch_size = 64
train_set = gdata.ArrayDataset(X_train, nd.array(y_train))
test_set = gdata.ArrayDataset(X_test, nd.array(y_test))
train_iter = gdata.DataLoader(train_set, batch_size, shuffle=True)
test_iter = gdata.DataLoader(test_set, batch_size)

loss = gloss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})
num_epochs = 100
# d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None,
#               None, trainer)

In [45]:
train(train_iter, test_iter, net, gloss.SoftmaxCrossEntropyLoss(), trainer, ctx, num_epochs, 10)

training on gpu(0)
epoch 1, loss 0.5812, train acc 0.686, test acc 0.721, time 24.7 sec
epoch 2, loss 0.5347, train acc 0.723, test acc 0.724, time 25.9 sec
epoch 3, loss 0.5126, train acc 0.738, test acc 0.744, time 25.9 sec
epoch 4, loss 0.4977, train acc 0.748, test acc 0.754, time 25.8 sec
epoch 5, loss 0.4851, train acc 0.756, test acc 0.757, time 25.8 sec
epoch 6, loss 0.4745, train acc 0.764, test acc 0.763, time 25.8 sec
epoch 7, loss 0.4648, train acc 0.770, test acc 0.753, time 25.8 sec
epoch 8, loss 0.4553, train acc 0.777, test acc 0.769, time 25.9 sec
epoch 9, loss 0.4472, train acc 0.782, test acc 0.772, time 25.7 sec
epoch 10, loss 0.4385, train acc 0.787, test acc 0.750, time 25.8 sec
epoch 11, loss 0.4113, train acc 0.804, test acc 0.780, time 25.7 sec
epoch 12, loss 0.4042, train acc 0.809, test acc 0.781, time 25.8 sec
epoch 13, loss 0.3978, train acc 0.812, test acc 0.774, time 25.7 sec
epoch 14, loss 0.3915, train acc 0.816, test acc 0.779, time 25.9 sec
epoch 15, 

In [46]:
net = nn.HybridSequential()
net.add(nn.Dense(512, activation='relu'))
net.add(gluon.nn.Dropout(0.25))
net.add(nn.Dense(2))
#     net.initialize(init.Normal(sigma=0.01))
net.initialize(init.Xavier(), ctx=ctx)
net.hybridize()

# net.initialize(init.Normal(sigma=0.01))

batch_size = 64
train_set = gdata.ArrayDataset(X_train, nd.array(y_train))
test_set = gdata.ArrayDataset(X_test, nd.array(y_test))
train_iter = gdata.DataLoader(train_set, batch_size, shuffle=True)
test_iter = gdata.DataLoader(test_set, batch_size)

loss = gloss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1, 'wd': 0.95})
num_epochs = 100
# d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None,
#               None, trainer)

In [47]:
train(train_iter, test_iter, net, gloss.SoftmaxCrossEntropyLoss(), trainer, ctx, num_epochs, 10)

training on gpu(0)
epoch 1, loss 0.6932, train acc 0.500, test acc 0.506, time 25.8 sec
epoch 2, loss 0.6932, train acc 0.500, test acc 0.506, time 25.8 sec
epoch 3, loss 0.6932, train acc 0.501, test acc 0.506, time 25.9 sec
epoch 4, loss 0.6932, train acc 0.500, test acc 0.494, time 26.0 sec
epoch 5, loss 0.6932, train acc 0.500, test acc 0.506, time 25.8 sec
epoch 6, loss 0.6932, train acc 0.502, test acc 0.506, time 25.9 sec
epoch 7, loss 0.6932, train acc 0.500, test acc 0.506, time 25.9 sec
epoch 8, loss 0.6932, train acc 0.502, test acc 0.506, time 25.8 sec
epoch 9, loss 0.6932, train acc 0.500, test acc 0.506, time 25.9 sec
epoch 10, loss 0.6932, train acc 0.501, test acc 0.506, time 25.8 sec
epoch 11, loss 0.6932, train acc 0.500, test acc 0.494, time 25.8 sec
epoch 12, loss 0.6931, train acc 0.502, test acc 0.506, time 25.8 sec
epoch 13, loss 0.6932, train acc 0.501, test acc 0.506, time 25.8 sec
epoch 14, loss 0.6932, train acc 0.502, test acc 0.506, time 25.9 sec
epoch 15, 

In [50]:
net = nn.HybridSequential()
net.add(nn.Dense(512, activation='relu'))
net.add(gluon.nn.Dropout(0.25))
net.add(nn.Dense(2))
#     net.initialize(init.Normal(sigma=0.01))
net.initialize(init.Xavier(), ctx=ctx)
net.hybridize()

# net.initialize(init.Normal(sigma=0.01))

batch_size = 64
train_set = gdata.ArrayDataset(X_train, nd.array(y_train))
test_set = gdata.ArrayDataset(X_test, nd.array(y_test))
train_iter = gdata.DataLoader(train_set, batch_size, shuffle=True)
test_iter = gdata.DataLoader(test_set, batch_size)

loss = gloss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1, 'wd': 0.001})
num_epochs = 100
# d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None,
#               None, trainer)

In [51]:
train(train_iter, test_iter, net, gloss.SoftmaxCrossEntropyLoss(), trainer, ctx, num_epochs, 10)

training on gpu(0)
epoch 1, loss 0.5865, train acc 0.681, test acc 0.672, time 25.7 sec
epoch 2, loss 0.5534, train acc 0.708, test acc 0.710, time 25.7 sec
epoch 3, loss 0.5426, train acc 0.717, test acc 0.717, time 25.6 sec
epoch 4, loss 0.5370, train acc 0.721, test acc 0.735, time 25.6 sec
epoch 5, loss 0.5350, train acc 0.722, test acc 0.731, time 25.8 sec
epoch 6, loss 0.5330, train acc 0.724, test acc 0.736, time 25.7 sec
epoch 7, loss 0.5317, train acc 0.725, test acc 0.738, time 25.7 sec
epoch 8, loss 0.5311, train acc 0.725, test acc 0.736, time 25.7 sec
epoch 9, loss 0.5299, train acc 0.726, test acc 0.722, time 25.6 sec
epoch 10, loss 0.5299, train acc 0.727, test acc 0.668, time 25.6 sec
epoch 11, loss 0.5085, train acc 0.742, test acc 0.743, time 25.8 sec
epoch 12, loss 0.5078, train acc 0.744, test acc 0.747, time 25.7 sec
epoch 13, loss 0.5068, train acc 0.743, test acc 0.744, time 25.8 sec
epoch 14, loss 0.5060, train acc 0.744, test acc 0.701, time 25.7 sec
epoch 15, 

In [55]:
def train(train_iter, test_iter, net, loss, trainer, ctx, num_epochs, lr_period):
    """Train and evaluate a model."""
    print('training on', ctx)
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, m, start = 0.0, 0.0, 0, 0, time.time()
        if epoch > 0 and epoch % lr_period == 0:
             trainer.set_learning_rate(trainer.learning_rate * .9)
        for i, batch in enumerate(train_iter):
            Xs, ys, batch_size = _get_batch(batch, ctx)
            ls = []
            with autograd.record():
                y_hats = [net(X) for X in Xs]
                ls = [loss(y_hat, y) for y_hat, y in zip(y_hats, ys)]
            for l in ls:
                l.backward()
            trainer.step(batch_size)
            train_l_sum += sum([l.sum().asscalar() for l in ls])
            n += sum([l.size for l in ls])
            train_acc_sum += sum([(y_hat.argmax(axis=1) == y).sum().asscalar()
                                  for y_hat, y in zip(y_hats, ys)])
            m += sum([y.size for y in ys])
        test_acc = evaluate_accuracy(test_iter, net, ctx)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / m, test_acc,
                 time.time() - start))


In [57]:
net = nn.HybridSequential()
net.add(nn.Dense(2048, activation='relu'))
net.add(gluon.nn.Dropout(0.25))
net.add(nn.Dense(512, activation='relu'))
net.add(nn.Dense(2))
#     net.initialize(init.Normal(sigma=0.01))
net.initialize(init.Xavier(), ctx=ctx)
net.hybridize()

# net.initialize(init.Normal(sigma=0.01))

batch_size = 64
train_set = gdata.ArrayDataset(X_train, nd.array(y_train))
test_set = gdata.ArrayDataset(X_test, nd.array(y_test))
train_iter = gdata.DataLoader(train_set, batch_size, shuffle=True)
test_iter = gdata.DataLoader(test_set, batch_size)

loss = gloss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1, 'wd': 0.001})
num_epochs = 100
# d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None,
#               None, trainer)

In [ ]:
train(train_iter, test_iter, net, gloss.SoftmaxCrossEntropyLoss(), trainer, ctx, num_epochs, 10)

training on gpu(0)
epoch 1, loss 0.5867, train acc 0.681, test acc 0.661, time 29.8 sec
epoch 2, loss 0.5526, train acc 0.708, test acc 0.732, time 29.1 sec
epoch 3, loss 0.5400, train acc 0.718, test acc 0.735, time 29.1 sec
epoch 4, loss 0.5345, train acc 0.722, test acc 0.715, time 28.9 sec
epoch 5, loss 0.5310, train acc 0.725, test acc 0.731, time 28.9 sec
epoch 6, loss 0.5281, train acc 0.727, test acc 0.745, time 29.1 sec
epoch 7, loss 0.5264, train acc 0.727, test acc 0.746, time 29.3 sec
epoch 8, loss 0.5257, train acc 0.728, test acc 0.740, time 29.4 sec
epoch 9, loss 0.5247, train acc 0.730, test acc 0.748, time 29.0 sec
epoch 10, loss 0.5234, train acc 0.729, test acc 0.741, time 29.2 sec
epoch 11, loss 0.5190, train acc 0.733, test acc 0.741, time 29.4 sec
epoch 12, loss 0.5182, train acc 0.734, test acc 0.744, time 29.4 sec
epoch 13, loss 0.5180, train acc 0.733, test acc 0.719, time 29.5 sec
epoch 14, loss 0.5165, train acc 0.735, test acc 0.744, time 29.2 sec
epoch 15, 